In [2]:
import numpy as np
import pandas as pd
import math


In [3]:
train = np.load('./dt_training/fashion_train.npy')

train_test = train[:, : -1]
y = train[:,-1]

print(train_test.shape, y.shape)

(10000, 784) (10000,)


In [4]:



def input_layer(X_train, y_train):
    n_feartures = X_train.shape[1]
    output_nodes = len(set(y_train))
    len_of_X_train = len(X_train)
    return n_feartures, output_nodes, len_of_X_train


def hidden_layer(X, n_feartures, nunbers_of_nodes_hidden, len_of_X_train):
    #first hidden layer

    W = np.random.randn(n_feartures, nunbers_of_nodes_hidden)

    b = np.random.randn(len_of_X_train, nunbers_of_nodes_hidden)

    z = X@W+b

    a = np.maximum(0, z)

    return a


#output layer
def output_layer(a, output_nodes, len_of_X_train):

    W = np.random.randn(a.shape[1], output_nodes)

    b = np.random.randn(len_of_X_train, output_nodes)

    z = a@W+b

    softmax = np.exp(z - np.max(z))/np.sum(np.exp(z - np.max(z)))
    preds = np.argmax(softmax, axis=1)

    return print(f'pred: {np.argmax(softmax, axis=1)}, shape: {softmax.shape}'), preds



In [27]:
X = train_test

n_feartures, output_nodes, len_of_X_train = input_layer(X, y)

a = hidden_layer(X, n_feartures, nunbers_of_nodes_hidden=40, len_of_X_train=len_of_X_train)

preds = output_layer(a, output_nodes, len_of_X_train)

np.unique(preds[1])

pred: [0 0 0 ... 0 0 0], shape: (10000, 5)


array([0, 4])